In [89]:
import numpy as np
import cv2

In [94]:
img = cv2.imread('/home/optolab/ur2020_workspace/src/telemove/src/example.png')
img = cv2.flip(img, 0)
img = img[0:900, 520:1650]

dst = cv2.imread('/home/optolab/ur2020_workspace/src/telemove/src/calibresult.png')
dst = cv2.flip(dst, 0)
dst = dst[0:900, 520:1650]

In [95]:
K = np.array([[1.06314155e+03, 0.00000000e+00, 9.62163918e+02],
              [0.00000000e+00, 1.06450171e+03, 5.40462082e+02],
              [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
D = np.array([[0.04147462, -0.03934306, -0.00013871, -0.00071449, -0.00889901]])


# trovate usando in calibrazione l'immagine non antidistorta
R = np.array([[ 0.99981412, -0.01682312, -0.00941902],
              [ 0.01644207,  0.99909738, -0.03916732],
              [ 0.01006943,  0.03900517,  0.99918827]])
t = np.array([[-33.05102197], [-15.29010071], [41.22994723]])

# trovate usando in calibrazione l'immagine antidistorta
Rd = np.array([[ 0.99983156, -0.01274002, -0.01321169],
               [ 0.01235797,  0.99951436, -0.02860628],
               [ 0.01356972,  0.02843819,  0.99950344]])
td = np.array([[-34.2646834], [-15.84329211], [42.18894322]])


def px2meters(pt, K, R, t):
    ''' Funzione di conversione tra pixel e metri per ottenere la posizione
    corretta in metri della posizione rilevata nell'immagine RGB-D. La Kinect
    prende (x,y) dall'RGB e (z) dal sensore di profondita', quindi deve essere
    calibrato per forza. Calibrazione fatta in MATLAB! Riporto qui le matrici '''

    # pt e' px
    
    K2 = np.linalg.inv(K)
    R2 = np.linalg.inv(R)
    pt = pt.T

    S = K2.dot(pt)
    N = S - t
    PT = R2.dot(N)
    XYZ = PT # in cm
    print('Point is: ' + str(XYZ))

    return XYZ

reference1, _ = cv2.projectPoints(np.array([[0.0, 0.0, 0.0]]), R, t, K, D)
reference1 = reference1.flatten()
print('Reference (distorted) is: ' + str(int(round(reference1[0]))) + ', ' + str(int(round(reference1[1]))))
reference1 = np.array([[int(round(reference1[0])), int(round(reference1[1])), 1.0]]) #px coordinates

reference2, _ = cv2.projectPoints(np.array([[0.0, 0.0, 0.0]]), Rd, td, K, D)
reference2 = reference2.flatten()
print('Reference (undistorted) is: ' + str(int(round(reference2[0]))) + ', ' + str(int(round(reference2[1]))))
reference2 = np.array([[int(round(reference2[0])), int(round(reference2[1])), 1.0]]) #px coordinates

Reference (distorted) is: 105, 143
Reference (undistorted) is: 94, 139


In [97]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
img = cv2.drawChessboardCorners(img, (9,6), corners2,ret)

grayD = cv2.cvtColor(dst,cv2.COLOR_BGR2GRAY)
retD, cornersD = cv2.findChessboardCorners(grayD, (9,6),None)
corners2D = cv2.cornerSubPix(grayD,cornersD,(11,11),(-1,-1),criteria)
dst = cv2.drawChessboardCorners(dst, (9,6), corners2D,retD)

In [98]:
cv2.imshow('Img',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
# visualizzo punti
#cv2.circle(img, (int(round(corners[1][0][0])),int(round(corners[1][0][1]))), 5, (0,255,0), thickness=-1, lineType=8, shift=0)
#cv2.imshow('image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [99]:
# creo vettore di punti scacchiera convertiti in intero, x-y
points = np.zeros((54,3))
pointsD = np.zeros((54,3))
for i in range(0,54):
    points[i][0] = round(corners2[i][0][0]) #x
    points[i][1] = round(corners2[i][0][1]) #y
    points[i][2] = 1.0 #z
    
    pointsD[i][0] = round(corners2D[i][0][0]) #x
    pointsD[i][1] = round(corners2D[i][0][1]) #y
    pointsD[i][2] = 1.0 #z

In [106]:
#IMMAGINE NON ANTIDISTORTA
ref = px2meters(reference1, K, R, t)
diff = []
for i in range(0,54):
    print('Difference between point ' + str(i-1) + ' and ' + str(i))
    pt = []
    pt = np.array([points[i]])
    PT = px2meters(pt, K, R, t)
    print('Difference [cm]: ' + str((PT-ref)*100)) # in cm
    D = (PT-ref)*100
    diff.append((D[0][0], D[1][0], D[2][0]))
    ref = PT

print(diff)

Point is: [[ 32.07894133]
 [ 12.79162469]
 [-41.08525351]]
Difference between point -1 and 0
Point is: [[ 32.07706046]
 [ 12.79165634]
 [-41.08523579]]
Difference [cm]: [[-0.18808673]
 [ 0.00316479]
 [ 0.00177192]]
[[-0.18808673]
 [ 0.00316479]
 [ 0.00177192]]
Difference between point 0 and 1
Point is: [[ 32.10151174]
 [ 12.79124491]
 [-41.08546614]]
Difference [cm]: [[ 2.44512754]
 [-0.04114231]
 [-0.02303498]]
[[ 2.44512754]
 [-0.04114231]
 [-0.02303498]]
Difference between point 1 and 2
Point is: [[ 32.12596302]
 [ 12.79083349]
 [-41.08569649]]
Difference [cm]: [[ 2.44512754]
 [-0.04114231]
 [-0.02303498]]
[[ 2.44512754]
 [-0.04114231]
 [-0.02303498]]
Difference between point 2 and 3
Point is: [[ 32.15042974]
 [ 12.79136062]
 [-41.08596364]]
Difference [cm]: [[ 2.44667212]
 [ 0.05271356]
 [-0.02671438]]
[[ 2.44667212]
 [ 0.05271356]
 [-0.02671438]]
Difference between point 3 and 4
Point is: [[ 32.17489646]
 [ 12.79188776]
 [-41.08623078]]
Difference [cm]: [[ 2.44667212]
 [ 0.0527135

In [107]:
# IMMAGINE ANTIDISTORTA
ref = px2meters(reference2, K, R, t)
diff = []
for i in range(0,54):
    print('Difference between point ' + str(i-1) + ' and ' + str(i))
    pt = []
    pt = np.array([pointsD[i]])
    PT = px2meters(pt, K, R, t)
    print('Difference [cm]: ' + str((PT-ref)*100)) # in cm
    D = (PT-ref)*100
    diff.append((D[0][0], D[1][0], D[2][0]))
    ref = PT
    
print(diff)

Point is: [[ 32.06853478]
 [ 12.78804452]
 [-41.08500888]]
Difference between point -1 and 0
Point is: [[ 32.06662302]
 [ 12.78619905]
 [-41.08491757]]
Difference [cm]: [[-0.19117589]
 [-0.18454695]
 [ 0.00913073]]
Difference between point 0 and 1
Point is: [[ 32.09108974]
 [ 12.78672618]
 [-41.08518472]]
Difference [cm]: [[ 2.44667212]
 [ 0.05271356]
 [-0.02671438]]
Difference between point 1 and 2
Point is: [[ 32.11460058]
 [ 12.78633058]
 [-41.08540621]]
Difference [cm]: [[ 2.35108418]
 [-0.03955992]
 [-0.02214902]]
Difference between point 2 and 3
Point is: [[ 32.13812687]
 [ 12.78687354]
 [-41.08566449]]
Difference [cm]: [[ 2.35262876]
 [ 0.05429596]
 [-0.02582842]]
Difference between point 3 and 4
Point is: [[ 32.16257815]
 [ 12.78646212]
 [-41.08589484]]
Difference [cm]: [[ 2.44512754]
 [-0.04114231]
 [-0.02303498]]
Difference between point 4 and 5
Point is: [[ 32.18611988]
 [ 12.78794364]
 [-41.08618992]]
Difference [cm]: [[ 2.35417334]
 [ 0.14815183]
 [-0.02950782]]
Difference